# Original DRIS

Diagnosis and Recommendation Integrated System (DRIS) implemented according to "Diagnosis and Recommendation Integrated System (DRIS)" by J.L Walworth and M.E. Sumner. 

Questions:

- Calculate DRIS to compare two populations or to compare each sample plant with the target population?
- Is it the mean of the ratios or the ratios of the mean? They are proportional, with the factor "number of plants"

In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

In [92]:
data_diagnosed_dict = {
    'P': [12,11,10, 9],
    'Mg': [2,2,3,3],
    'N': [1,2,1,2],
    'Ca': [3,4,3,3],
    'Mn': [4,5,4,2]
}

data_optimum_dict = {
    'p': [12,11,10],
    'mg': [2,1,3],
    'n': [2,1,1],
    'ca': [4,4,3],
    'mn': [5,4,4],
}

df_diagnosed = pd.DataFrame(data_diagnosed_dict)
df_optimum = pd.DataFrame(data_optimum_dict)

## Calculate $f(A/B)$ 

\begin{align}
    f(A/B) = 
    \begin{cases}
    \biggl(\frac{A/B}{a/b} - 1\biggr)\frac{1000}{CV} & A/B \ge a/b\\
    \biggl(1 - \frac{a/b}{A/B}\biggr)\frac{1000}{CV} & A/B < a/b
    \end{cases}
\end{align}

- $A/B$ are the ratios of the means two elements (A and B) in the tissue of the pant being diagnosed.
- $a/b$ are the ratios of the means for the optimal (high yield) population. 
- CV is the coefficient of variation of the optimal values $a$ and $b$: $CV = \sigma/\mu$ 

**Comments:**
- Is a/b (resp. A/B) the mean of the ratios or the ration of the means? This is not clear to me!!!
- In the poplication of Jones the value is 100 not 1000, but in subsequent publications it is 1000.
- Is the CV (coefficient of variation) in percent or not (in the publication of jones it certainly is)

Calculate ratios A/B and a/b 

In [93]:
def calculate_ratios(df):
    '''Calculate the rations of every combination'''
    ratios = {}
    for col1, col2 in combinations(df.columns, 2):
        ratio_name1 = f"{col1}/{col2}"
        ratios[ratio_name1] = df[col1] / df[col2]
    return pd.DataFrame(ratios)

df_optimum_ratios = calculate_ratios(df_optimum)
df_diagnosed_ratios = calculate_ratios(df_diagnosed)

df_diagnosed_ratios.head()

,P/Mg,P/N,P/Ca,P/Mn,Mg/N,Mg/Ca,Mg/Mn,N/Ca,N/Mn,Ca/Mn
0,6.000000,12.0,4.000000,3.0,2.0,0.666667,0.50,0.333333,0.25,0.75
1,5.500000,5.5,2.750000,2.2,1.0,0.500000,0.40,0.500000,0.40,0.80
2,3.333333,10.0,3.333333,2.5,3.0,1.000000,0.75,0.333333,0.25,0.75
3,3.000000,4.5,3.000000,4.5,1.5,1.000000,1.50,0.666667,1.00,1.50


### Calculate coefficient of variation of the optimum values

In [94]:
def calculate_CV(df_ratios):
    cv_dict = {}
    for ratio_name, ratio_values in df_ratios.items():
        mean_ratio = ratio_values.mean()
        std_ratio = ratio_values.std()
        cv = (std_ratio / mean_ratio) * 100  # percentage or not? properbly percentage
        cv_dict[ratio_name] = [cv]
    return pd.DataFrame(cv_dict)

df_optimum_CV = calculate_CV(df_optimum_ratios)

### Calculate rations of the mean

In [95]:
df_op = pd.DataFrame(df_optimum.mean()).T
df_optimum_mean_ratios = calculate_ratios(df_op)

df_di = pd.DataFrame(df_diagnosed.mean()).T
df_diagnosed_mean_ratios = calculate_ratios(df_di)

df_diagnosed_mean_ratios.head()

,P/Mg,P/N,P/Ca,P/Mn,Mg/N,Mg/Ca,Mg/Mn,N/Ca,N/Mn,Ca/Mn
0,4.2,7.0,3.230769,2.8,1.666667,0.769231,0.666667,0.461538,0.4,0.866667


In [96]:
df_optimum_mean_ratios.head()

,p/mg,p/n,p/ca,p/mn,mg/n,mg/ca,mg/mn,n/ca,n/mn,ca/mn
0,5.5,8.25,3.0,2.538462,1.5,0.545455,0.461538,0.363636,0.307692,0.846154


### Calculate nutrient function

In [97]:
def f(df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV):
    names = df_diagnosed_mean_ratios.columns
    diagnosed_ratios = df_diagnosed_mean_ratios.to_numpy()
    optimum_mean_ratios = df_optimum_mean_ratios.to_numpy()
    optimum_CV = df_optimum_CV.to_numpy()

    def f_single(diagnosed_ratio, optimum_mean_ratio, CV):
        if diagnosed_ratio == 0:
            return float('inf')  # Return inf or some large number to handle zero division
        if diagnosed_ratio >= optimum_mean_ratio:
            f = ((diagnosed_ratio / optimum_mean_ratio) - 1) * 1000 / CV
        else:
            f = (1 - (optimum_mean_ratio / diagnosed_ratio)) * 1000 / CV
        return [f]

    f_dict = dict()
    for i in range(len(names)):
        f_dict[f"f({names[i]})"] = f_single(diagnosed_ratios[0, i], optimum_mean_ratios[0, i], optimum_CV[0, i])
    return pd.DataFrame(f_dict)


# Example usage
# Assuming df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV are defined DataFrames with similar structure
df_f = f(df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV)
df_f.head()


,f(P/Mg),f(P/N),f(P/Ca),f(P/Mn),f(Mg/N),f(Mg/Ca),f(Mg/Mn),f(N/Ca),f(N/Mn),f(Ca/Mn)
0,-5.390159,-6.074429,7.958325,14.573487,1.603751,6.26677,8.083535,7.637626,10.392305,1.557672


## Calculate DRIS index


$\mathrm{A_{index}} = \frac{1}{z} \bigl[f(A/B) + f(A/C) + f(A/D) + \cdots + f(A/N)\bigr]$

In [98]:
def create_index_string(index_element, df_diagnosed):
    ''' Create string representing the the equation to calculate the DRIS index '''
    elements = df_diagnosed.columns
    result_string = f'I_{elements[index_element]} = 1/{(len(elements)-1)} ('
    for i, element in enumerate(elements):
        if index_element < i:
            result_string += (f' + f({elements[index_element]}/{element})')
        elif index_element > i:
            result_string += (f' - f({element}/{elements[index_element]})')
    return result_string + ')'

result_string = create_index_string(2,df_diagnosed)
result_string

'I_N = 1/4 ( - f(P/N) - f(Mg/N) + f(N/Ca) + f(N/Mn))'

In [99]:
def calculate_index_value(index_element, df_diagnosed, df_f):
    ''' Calculates the DRIS index '''
    f_dict = df_f.to_dict('index')[0]
    elements = df_diagnosed.columns
    result = 0
    for i, element in enumerate(elements):
        if index_element < i:
            result += f_dict[f'f({elements[index_element]}/{element})']
        elif index_element > i:
            result -= f_dict[f'f({element}/{elements[index_element]})']
    return result/(len(elements)-1)

In [100]:
def calculate_all_index_values(df_diagnosed, df_f):
    """
    Calculate the index values for each element in the input DataFrame.
    
    Parameters:
        df_diagnosed (pd.DataFrame): DataFrame where each column represents a diagnosed nutrient.
        df_f (pd.DataFrame): DataFrame with calculated f(A/B) or f(B/A) values for each element ratio.
        
    Returns:
        pd.DataFrame: DataFrame with index values for each element, where each row corresponds to an element.
    """
    elements = df_diagnosed.columns
    results_dict = {}
    
    # Calculate index value for each element and store in results_dict
    for i, element in enumerate(elements):
        # Assuming calculate_index_value is a function that calculates the index for a single element
        index_value = calculate_index_value(i, df_diagnosed, df_f)
        results_dict[element] = index_value
    
    # Convert results_dict to a DataFrame with the specified format
    results_df = pd.DataFrame(list(results_dict.items()), columns=["Element", "I_DRIS"])
    results_df.set_index("Element", inplace=True)
    results_df.index.name = None

    
    return results_df

# Example usage
# df_diagnosed and df_f should be DataFrames with appropriate data for the calculation
DRIS_indices = calculate_all_index_values(df_diagnosed, df_f)
DRIS_indices.T.head()


,P,Mg,N,Ca,Mn
I_DRIS,2.766806,5.336054,5.625152,-5.076262,-8.65175


## Putting it together

In [101]:
def calculate_DRIS_index(df_diagnosed, df_optimum):
    # calculate ratios
    df_optimum_ratios = calculate_ratios(df_optimum)
    # df_diagnosed_ratios = calculate_ratios(df_diagnosed)
    # calculate CV
    df_optimum_CV = calculate_CV(df_optimum_ratios)
    # calculate mean _ratios
    df_op = pd.DataFrame(df_optimum.mean()).T
    df_optimum_mean_ratios = calculate_ratios(df_op)
    df_di = pd.DataFrame(df_diagnosed.mean()).T
    df_diagnosed_mean_ratios = calculate_ratios(df_di)
    # calculate f
    df_f = f(df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV)
    # calculate indices
    DRIS_indices = calculate_all_index_values(df_diagnosed, df_f)
    return DRIS_indices

DRIS = calculate_DRIS_index(df_diagnosed, df_optimum)
DRIS.T.head()

optimum_stds = df_optimum.std()  # Compute standard deviations

# Convert Series to DataFrame and set index
optimum_stds_df = pd.DataFrame(optimum_stds, columns=["Standard Deviation optimum"])
optimum_stds_df.index = DRIS.T.columns


df_nutrients = pd.concat([DRIS.T, optimum_stds_df.T])

df_nutrients.head()

,P,Mg,N,Ca,Mn
I_DRIS,2.766806,5.336054,5.625152,-5.076262,-8.65175
Standard Deviation optimum,1.000000,1.000000,0.577350,0.577350,0.57735


## Nurtional Balance Index

In [102]:
NBI = df_nutrients.loc['I_DRIS'].abs().sum()
print(f"NBI = {NBI}")
NBI_m = NBI/len(df_nutrients.columns)
print(f"NBI_m = {NBI_m}")


NBI = 27.456024105751258
NBI_m = 5.4912048211502515


## Range

In [103]:
def operation(column):
    I = column.iloc[0]
    sd = column.iloc[1]
    if I < -4/3*sd:
        return "deficiency"
    elif I < -2/3*sd:
        return "tendency to deficiency"
    elif I < 2/3*sd:
        return "sufficient"
    elif I < 4/3*sd:
        return "tendency to excess"
    else:
        return "excess"
    
df_nutrients.loc['Interpretation DRIS'] = df_nutrients.apply(operation, axis=0)
df_nutrients

,P,Mg,N,Ca,Mn
I_DRIS,2.766806,5.336054,5.625152,-5.076262,-8.65175
Standard Deviation optimum,1.0,1.0,0.57735,0.57735,0.57735
Interpretation DRIS,excess,excess,excess,deficiency,deficiency
